In [1]:
from pykeen.pipeline import pipeline
from pykeen.datasets import Nations, get_dataset
import torch
from pykeen.evaluation import evaluate, RankBasedEvaluator
from pykeen.metrics.ranking import HitsAtK
import pandas as pd


import logging
from pathlib import Path

import click
import more_click
import torch
from pykeen.evaluation import RankBasedEvaluator
from pykeen.losses import NSSALoss,CrossEntropyLoss
from pykeen.models.inductive import InductiveNodePiece, InductiveNodePieceGNN
from pykeen.trackers import ConsoleResultTracker, WANDBResultTracker, FileResultTracker
from pykeen.training import SLCWATrainingLoop
from pykeen.typing import TESTING, TRAINING, VALIDATION
from pykeen.utils import resolve_device, set_random_seed
from torch.optim import Adam


from pykeen.metrics.ranking import HitsAtK

from pathlib import Path

from pykeen.datasets.inductive.base import DisjointInductivePathDataset
from pykeen.datasets.base import PathDataset, Dataset
from typing_extensions import Literal
import os
from pykeen.hpo import hpo_pipeline
from pykeen.triples import TriplesFactory
from pykeen.models import InductiveNodePiece, TransE, RGCN, ConvE
from pykeen.typing import TESTING, TRAINING, VALIDATION

import time

import platform

import sys

import cpuinfo

import psutil

import subprocess

import zipfile

seed = 1234

In [2]:
def show_metrics(dictionary,model_name,csv_name):
    for key in dictionary.keys():
        print(key)
        df = pd.DataFrame(dictionary[key])
        df.to_csv(f"{model_name}/{model_name}_{csv_name}_{key}.csv")
        print(df)

In [3]:
DATA_TYPE = "_transductive.tsv"
TRAIN_PATH = "MSCallGraph_0_train" + DATA_TYPE
TEST_PATH = "MSCallGraph_0_test" + DATA_TYPE
VALIDATE_PATH = "MSCallGraph_0_validation" + DATA_TYPE

In [4]:
dataset = PathDataset(training_path = TRAIN_PATH,
                     testing_path = TEST_PATH,
                     validation_path = VALIDATE_PATH,
                      eager = True,
                     create_inverse_triples = True
                     )


In [5]:
model_name = 'convE_0_transductive'

In [6]:
tracker = ConsoleResultTracker()

In [7]:
loss = NSSALoss() #used by RotatE and NodePiece
embedding_dim = 200
batch_size = 512

In [8]:
model = ConvE(
        triples_factory=dataset.training,
        random_seed = seed,
        loss = loss,
        embedding_dim = embedding_dim
    ).to(resolve_device())
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Space occupied: {model.num_parameter_bytes} bytes")

Number of parameters: 3561572
Space occupied: 14246288 bytes


In [9]:
directory = model_name

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f'Directory {directory} created successfully!')
else:
    print(f'Directory {directory} already exists.')

Directory convE_0_transductive created successfully!


In [10]:
learning_rate = 1e-3
optimizer = Adam(params=model.parameters(), lr=learning_rate)
num_epochs = 2000
patience = 20

In [11]:
metrics = ['meanreciprocalrank', HitsAtK(1),
                 HitsAtK(3), HitsAtK(5), HitsAtK(10)]

train_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
valid_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
test_evaluator = RankBasedEvaluator(
        metrics = metrics,
        add_defaults=False
    )

In [12]:
from pykeen.stoppers import EarlyStopper

stopper = EarlyStopper(
    model = model,
    metric='meanreciprocalrank',
    patience=patience,
    frequency=1,
    evaluator = valid_evaluator,
    training_triples_factory = dataset.training,
    evaluation_triples_factory = dataset.validation,
    result_tracker = tracker

)



In [13]:
# default training regime is negative sampling (SLCWA)
# you can also use the 1-N regime with the LCWATrainingLoop
# the LCWA loop does not need negative sampling kwargs, but accepts label_smoothing in the .train() method
training_loop = SLCWATrainingLoop(
        triples_factory=dataset.training,
        model=model,
        result_tracker=tracker,
        optimizer=optimizer
)

In [14]:
training_start = time.time()
train_epoch =  training_loop.train(
        triples_factory=dataset.training,
        num_epochs=num_epochs,
        stopper = stopper,
        batch_size = batch_size
        
    )
training_duration = time.time() - training_start

Training epochs on cuda:0:   0%|          | 0/2000 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/40 [00:00<?, ?batch/s]

Step: 1
Metric: loss = 4.317149150371551
Step: 1
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.0009374341749810621
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.0003996047250265193
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.0006685194500037908
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.0009374342043884099
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.0003996047598775477
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.0006685194675810635
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0009374341749810621
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.0003996047250265193
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.0006685194500037908
Metric: validation.head.optimistic.hits_at_1 = 0.0003891050583657588
Metric: validation.tail.optimistic.hits_at_1 = 0.0
Metric: validation.both.optimistic.hits_at_1 = 0.00019455

Training batches on cuda:0:   0%|          | 0/40 [00:00<?, ?batch/s]

Step: 2
Metric: loss = 3.917000389099121
Step: 2
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.0006057029239903874
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.00044960004637702183
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.0005276514851837046
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.0006057029240764678
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.00044960001832805574
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.0005276515148580074
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0006057029239903874
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.00044960004637702183
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.0005276514851837046
Metric: validation.head.optimistic.hits_at_1 = 0.0
Metric: validation.tail.optimistic.hits_at_1 = 0.0
Metric: validation.both.optimistic.hits_at_1 = 0.0
Metric: validation.he

...

Step: 107
Metric: loss = 0.006874509563203901
Step: 107
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.050762942175918736
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.16156437816034322
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.10616366016813099
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.05076294019818306
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.16156437993049622
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.10616365820169449
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.050762942175918736
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.16156437816034322
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.10616366016813099
Metric: validation.head.optimistic.hits_at_1 = 0.015953307392996108
Metric: validation.tail.optimistic.hits_at_1 = 0.088715953307393
Metric: validation.both.optimistic.hits_at_1 = 0.0523

Training batches on cuda:0:   0%|          | 0/40 [00:00<?, ?batch/s]

Step: 108
Metric: loss = 0.00728172471281141
Step: 108
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.05147194661907596
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.1645632578786383
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.10801760224885712
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.05147194489836693
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.16456326842308044
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.10801759362220764
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.05147194661907596
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.1645632578786383
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.10801760224885712
Metric: validation.head.optimistic.hits_at_1 = 0.01634241245136187
Metric: validation.tail.optimistic.hits_at_1 = 0.08793774319066149
Metric: validation.both.optimistic.hits_at_1 = 0.05214007

Training batches on cuda:0:   0%|          | 0/40 [00:00<?, ?batch/s]

Step: 109
Metric: loss = 0.006510941160377115
Step: 109
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.05318235184285184
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.17074371689256207
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.11196303436770699
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.05318235233426094
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.17074371874332428
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.11196303367614746
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.05318235184285184
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.17074371689256207
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.11196303436770699
Metric: validation.head.optimistic.hits_at_1 = 0.018287937743190662
Metric: validation.tail.optimistic.hits_at_1 = 0.09533073929961089
Metric: validation.both.optimistic.hits_at_1 = 0.0568

In [15]:
torch.save(model,f"{model_name}/model.pth")
model = torch.load(f"{model_name}/model.pth")

In [16]:
print("Train error per epoch:")
df = pd.DataFrame(train_epoch)
print(df)
df.to_csv(f"{model_name}/{model_name}_train_error_per_epoch.csv")

Train error per epoch:
            0
0    4.317149
1    3.917000
2    3.541684
3    3.148534
4    2.751854
..        ...
104  0.007726
105  0.007741
106  0.006875
107  0.007282
108  0.006511

[109 rows x 1 columns]


In [17]:
training_evaluation_start = time.time()
# train
print("Train error")
show_metrics(train_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.training.mapped_triples,
        additional_filter_triples=[
        dataset.training.mapped_triples,
    ]
    ).to_dict(),model_name,'train_metrics')
training_evaluation_duration = time.time() - training_evaluation_start

Train error


Evaluating on cuda:0:   0%|          | 0.00/10.4k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.637983   0.637983     0.637983
hits_at_1                     0.479723   0.479723     0.479723
hits_at_3                     0.760235   0.760235     0.760235
hits_at_5                     0.836721   0.836721     0.836721
hits_at_10                    0.921877   0.921877     0.921877
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.894725   0.894725     0.894725
hits_at_1                     0.849822   0.849822     0.849822
hits_at_3                     0.926693   0.926693     0.926693
hits_at_5                     0.951642   0.951642     0.951642
hits_at_10                    0.974280   0.974280     0.974280
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.766354   0.766354     0.766354
hits_at_1                     0.664772   0.664772     0.664772
hits_at_3                     0.843464  

In [18]:
validation_evaluation_start = time.time()
# validation
print("Validation error")
show_metrics(valid_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.validation.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples
        ],
    ).to_dict(),model_name,'validation_metrics')
validation_evaluation_duration = time.time() - validation_evaluation_start

Validation error


Evaluating on cuda:0:   0%|          | 0.00/2.57k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.053806   0.053806     0.053806
hits_at_1                     0.019844   0.019844     0.019844
hits_at_3                     0.045914   0.045914     0.045914
hits_at_5                     0.068872   0.068872     0.068872
hits_at_10                    0.110117   0.110117     0.110117
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.175335   0.175335     0.175335
hits_at_1                     0.105447   0.105447     0.105447
hits_at_3                     0.202724   0.202724     0.202724
hits_at_5                     0.250195   0.250195     0.250195
hits_at_10                    0.300778   0.300778     0.300778
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.114571   0.114571     0.114571
hits_at_1                     0.062646   0.062646     0.062646
hits_at_3                     0.124319  

In [19]:
testing_evaluation_start = time.time()
# result on the test set
print("Test error")
show_metrics(test_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.testing.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples,
            dataset.validation.mapped_triples,
        ],
    ).to_dict(),model_name,'test_metrics')
testing_evaluation_duration = time.time() - testing_evaluation_start

Test error


Evaluating on cuda:0:   0%|          | 0.00/3.27k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.067490   0.067490     0.067490
hits_at_1                     0.029052   0.029052     0.029052
hits_at_3                     0.063303   0.063303     0.063303
hits_at_5                     0.086544   0.086544     0.086544
hits_at_10                    0.140673   0.140673     0.140673
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.193652   0.193652     0.193652
hits_at_1                     0.125076   0.125076     0.125076
hits_at_3                     0.215902   0.215902     0.215902
hits_at_5                     0.257187   0.257187     0.257187
hits_at_10                    0.322630   0.322630     0.322630
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.130571   0.130571     0.130571
hits_at_1                     0.077064   0.077064     0.077064
hits_at_3                     0.139602  

In [20]:
infodict = {}
infodict['device'] = model.device
infodict['parameters bytes'] = model.num_parameter_bytes
infodict['number parameters'] = model.num_parameters
infodict['training duration'] = training_duration
infodict['training evaluation duration'] = training_evaluation_duration
infodict['validation evaluation duration'] = validation_evaluation_duration
infodict['testing evaluation duration'] = testing_evaluation_duration
infodict["Operating system name"] = platform.system()
infodict["Operating system version"] = platform.release()
infodict["Processor architecture"] = platform.machine()
infodict["Python version"] = sys.version
infodict["Processor model name"] = cpuinfo.get_cpu_info()['brand_raw']
infodict['Number cpu cores'] = os.cpu_count()
infodict["Total physical memory"] = psutil.virtual_memory().total

In [21]:

output = subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv'])
output = output.decode('utf-8')  # convert byte string to regular string

# split output into rows and remove header row
rows = output.strip().split('\n')[1:]

# extract GPU names from each row
gpu_names = []
for row in rows:
    name = row.strip()
    gpu_names.append(name)

infodict['GPU'] = gpu_names[0]


In [22]:
infodict['loss'] = NSSALoss
infodict['embedding_dim'] = embedding_dim
infodict['learning_rate'] = learning_rate
infodict['optimizer'] = Adam
infodict['num_epochs'] = num_epochs
infodict['patience'] = patience
infodict['batch size'] = batch_size

In [23]:
info_df = pd.DataFrame(columns=['name','value'], data = infodict.items())
info_df.to_csv(f"{model_name}/{model_name}_information.csv")
print(info_df)

                              name  \
0                           device   
1                 parameters bytes   
2                number parameters   
3                training duration   
4     training evaluation duration   
5   validation evaluation duration   
6      testing evaluation duration   
7            Operating system name   
8         Operating system version   
9           Processor architecture   
10                  Python version   
11            Processor model name   
12                Number cpu cores   
13           Total physical memory   
14                             GPU   
15                            loss   
16                   embedding_dim   
17                   learning_rate   
18                       optimizer   
19                      num_epochs   
20                        patience   
21                      batch size   

                                                value  
0                                              cuda:0  
1            

In [24]:
def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file))

folder_path = model_name
output_path = f'{model_name}.zip'

zip_folder(folder_path, output_path)